<a href="https://colab.research.google.com/github/Amisha1019/Gen-AI-Customer-Service-Bot-Internship-Task/blob/main/Task_3_of_Gen_Ai_chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install --upgrade pandas numpy sentence-transformers faiss-cpu streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 141.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 134.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 166.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 150.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the 

In [7]:
!pip install spacy en_core_web_sm

In [8]:
import pandas as pd
import numpy as np
import os
import glob
import faiss
import pickle
import xml.etree.ElementTree as ET
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'faiss'

In [8]:
DATA_DIR = '/content'
INDEX_PATH = 'state_faiss.index'
DB_PATH = 'state_db.pkl'
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'

In [16]:
def parse_state_xml(file_path):
    """
    Parses a single state XML file to extract Question-Answer pairs and Focus.
    """
    qa_list = []
    # try: # Temporarily removed for detailed traceback
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Check for namespaces
    namespace = None
    if '}' in root.tag:
        namespace = root.tag.split('}')[0][1:]

    def find_element(element_name):
        if namespace:
            return root.find(f'{{{namespace}}}{element_name}')
        return root.find(element_name)

    question_element = find_element('Question')
    answer_element = find_element('Answer')
    focus_element = find_element('Focus')

    question = question_element.text.strip() if question_element is not None and question_element.text is not None else "N/A"
    answer = answer_element.text.strip() if answer_element is not None and answer_element.text is not None else "N/A"
    focus = focus_element.text.strip() if focus_element is not None and focus_element.text is not None else "N/A"


    if question != "N/A" and answer != "N/A":
         qa_list.append({
            'Question': question,
            'Answer': answer,
            'Focus': focus
        })
    elif question == "N/A":
        print(f"Warning: Missing 'Question' element in {file_path}")
    elif answer == "N/A":
        print(f"Warning: Missing 'Answer' element in {file_path}")
    elif focus == "N/A":
        print(f"Warning: Missing 'Focus' element in {file_path}")

    # except Exception as e: # Temporarily removed for detailed traceback
        # print(f"Error parsing {file_path}: {e}")

    return qa_list

In [17]:
def create_knowledge_base():
    """
    Loads data, creates embeddings, and builds the FAISS index.
    """
    print("Starting Knowledge Base Creation...")


    xml_files = glob.glob(os.path.join(DATA_DIR, '*.xml'))
    if not xml_files:
        print(f"ERROR: No XML files found in {DATA_DIR}. Please check the path.")
        return

    all_qa_data = []
    for xml_file in xml_files:
        all_qa_data.extend(parse_state_xml(xml_file))

    qa_df = pd.DataFrame(all_qa_data)


    qa_df = qa_df.dropna(subset=['Question', 'Answer'])
    qa_df = qa_df.drop_duplicates(subset=['Answer']).reset_index(drop=True)

    print(f"Total cleaned Q&A pairs loaded: {len(qa_df):,}")
    if qa_df.empty:
        print("No data found after cleaning. Exiting.")
        return

    print(f"Loading Sentence Transformer model: {EMBEDDING_MODEL}...")
    model = SentenceTransformer(EMBEDDING_MODEL)


    questions = qa_df['Question'].tolist()
    print("Generating embeddings for questions...")
    embeddings = model.encode(questions, show_progress_bar=True, convert_to_numpy=True)


    print("Building FAISS index...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    print(f"FAISS index contains {index.ntotal} vectors.")


    print(f"Saving FAISS index to {INDEX_PATH}...")
    faiss.write_index(index, INDEX_PATH)

    print(f"Saving knowledge base (DataFrame) to {DB_PATH}...")
    with open(DB_PATH, 'wb') as f:
        pickle.dump(qa_df, f)

    print("Knowledge Base Creation Complete.")

create_knowledge_base()

Starting Knowledge Base Creation...


NameError: name 'glob' is not defined

In [19]:
import os
import glob

DATA_DIR = '/content/0000001_2.xml'

xml_files = glob.glob(os.path.join(DATA_DIR, '*.xml'))

if not xml_files:
    print(f"No XML files found in {DATA_DIR}.")
else:
    for xml_file in xml_files:
        print(f"--- Content of {xml_file} ---")
        try:
            with open(xml_file, 'r') as f:
                print(f.read())
        except Exception as e:
            print(f"Error reading {xml_file}: {e}")
        print("-" * (len(xml_file) + 15))

No XML files found in /content/0000001_2.xml.


In [5]:
# This cell contained a duplicate function definition with syntax errors and has been removed.
# The correct function `create_knowledge_base` is defined in a previous cell.

Now that the knowledge base is created, we can implement a function to query it. This function will take a user query, embed it using the same model, and then use the FAISS index to find the most similar questions in the knowledge base. Finally, it will retrieve and display the corresponding answers.

In [7]:
import faiss
import pickle
import pandas as pd
from sentence_transformers import SentenceTransformer

def load_knowledge_base():
    """
    Loads the FAISS index and the knowledge base (DataFrame) from disk.
    """
    print("Loading knowledge base and index...")
    index = faiss.read_index(INDEX_PATH)
    with open(DB_PATH, 'rb') as f:
        qa_df = pickle.load(f)
    print("Knowledge base and index loaded.")
    return index, qa_df

def query_knowledge_base(query, index, qa_df, model, k=5):
    """
    Queries the knowledge base with a given query.

    Args:
        query (str): The user's query.
        index (faiss.Index): The FAISS index.
        qa_df (pd.DataFrame): The DataFrame containing the Q&A pairs.
        model (SentenceTransformer): The sentence embedding model.
        k (int): The number of nearest neighbors to retrieve.

    Returns:
        list: A list of dictionaries containing the top k results (Question, Answer, Score).
    """

    query_embedding = model.encode([query], convert_to_numpy=True)


    distances, indices = index.search(query_embedding, k)


    results = []
    for i in range(k):
        qa_index = indices[0][i]
        distance = distances[0][i]
        result = {
            'Question': qa_df.loc[qa_index, 'Question'],
            'Answer': qa_df.loc[qa_index, 'Answer'],
            'Focus': qa_df.loc[qa_index, 'Focus'],
            'Score': 1 - (distance / index.reconstruct(0).dot(index.reconstruct(0))) # Cosine similarity from L2 distance
        }
        results.append(result)

    return results


try:
    faiss_index, knowledge_base_df = load_knowledge_base()
    embedding_model = SentenceTransformer(EMBEDDING_MODEL)
except FileNotFoundError:
    print("Knowledge base files not found. Please run the knowledge base creation step first.")
    faiss_index = None
    knowledge_base_df = None
    embedding_model = None


if faiss_index is not None and knowledge_base_df is not None and embedding_model is not None:
    user_query = "What are the symptoms of diabetes?"
    print(f"\nSearching for: '{user_query}'")
    search_results = query_knowledge_base(user_query, faiss_index, knowledge_base_df, embedding_model)

    print("\nSearch Results:")
    for i, result in enumerate(search_results):
        print(f"--- Result {i+1} ---")
        print(f"Question: {result['Question']}")
        print(f"Answer: {result['Answer']}")
        print(f"Focus: {result['Focus']}")
        print(f"Score: {result['Score']:.4f}")

Loading knowledge base and index...


RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/third-party/faiss/faiss/impl/io.cpp:69: Error: 'f' failed: could not open state_faiss.index for reading: No such file or directory

In [20]:
import streamlit as st
import faiss
import pickle
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
import os

In [22]:
INDEX_PATH = 'state_faiss.index'
DB_PATH = 'state_db.pkl'

def load_rag_components():
    """Load the Sentence Transformer model, FAISS index, and Q&A database."""
    try:

        with open('model_name.txt', 'r') as f:
            model_name = f.read().strip()


        model = SentenceTransformer(model_name)
        index = faiss.read_index(INDEX_PATH)
        qa_df = pd.read_pickle(DB_PATH)

        print("RAG components loaded successfully.")
        return model, index, qa_df
    except Exception as e:
        print(f"Error loading RAG components. Did you run knowledge_base.py first? Error: {e}")
        return None, None, None

In [23]:
def load_ner_model():
    """Load the SpaCy model for Named Entity Recognition."""
    try:

        nlp = spacy.load("en_core_web_sm")
        print("SpaCy NER model loaded.")
        return nlp
    except Exception as e:
        print(f"Could not load SpaCy model. NER will be disabled. Error: {e}")
        return None

In [24]:
def semantic_search(query, model, index, qa_df, k=3):
    """Encodes the query and searches the FAISS index for the top k matches."""

    query_embedding = model.encode([query])

    faiss.normalize_L2(query_embedding)
    query_embedding = query_embedding.astype('float32')


    distances, indices = index.search(query_embedding, k)


    results = []
    for i in range(k):
        idx = indices[0][i]
        score = distances[0][i]

        if idx >= 0:
            results.append({
                'score': score,
                'Question': qa_df.iloc[idx]['Question'],
                'Answer': qa_df.iloc[idx]['Answer'],
                'Focus': qa_df.iloc[idx]['Focus']
            })
    return results

In [25]:
def perform_ner(query, nlp):
    """Identifies medical entities (simulated) in the user query."""
    if nlp is None:
        return []

    doc = nlp(query)
    entities = []


    for ent in doc.ents:

        if ent.label_ in ['PERSON', 'ORG', 'NORP', 'PRODUCT', 'EVENT']:
            entities.append((ent.text, ent.label_))


    medical_keywords = {
        'symptoms': ['pain', 'fever', 'tired', 'cough', 'bleeding'],
        'diseases': ['cancer', 'leukemia', 'glaucoma', 'myeloid', 'lymphoblastic'],
        'treatments': ['chemotherapy', 'radiation', 'transplant', 'surgery', 'tyrosine kinase']
    }

    query_lower = query.lower()
    for entity_type, keywords in medical_keywords.items():
        for keyword in keywords:
            if keyword in query_lower and (keyword, entity_type.upper()) not in entities:
                 entities.append((keyword.capitalize(), entity_type.upper()))

    return entities

In [26]:
st.set_page_config(page_title="state Medical Chatbot 🩺", layout="wide")
st.title("state Medical Chatbot 🩺")
st.markdown("---")


model, index, qa_df = load_rag_components()
nlp = load_ner_model()

if model and index and not qa_df.empty:
    if "messages" not in st.session_state:
        st.session_state.messages = [
            {"role": "assistant", "content": "Hello! I am a medical question-answering bot powered by the state dataset. Ask me about diseases, symptoms, or treatments."}
        ]


    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

2025-10-30 08:42:02.082 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:02.083 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:02.235 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-30 08:42:02.236 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:02.236 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:02.237 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:02.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

Error loading RAG components. Did you run knowledge_base.py first? Error: [Errno 2] No such file or directory: 'model_name.txt'
SpaCy NER model loaded.


In [27]:
if prompt := st.chat_input("Ask a medical question..."):

        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Searching and generating response..."):


                search_results = semantic_search(prompt, model, index, qa_df, k=3)
                best_match = search_results[0]


                entities = perform_ner(prompt, nlp)



                # A. NER Display
                entity_report = ""
                if entities:
                    entity_report += "**Identified Medical Entities:** "
                    entity_report += ", ".join([f"**{text}** (*{label}*)" for text, label in entities])

                # B. Retrieval Summary & Answer
                if best_match['score'] > 0.5:

                    answer_content = f"### Answer (Source: {best_match['Focus']})\n{best_match['Answer']}\n\n"
                    answer_content += f"*Retrieval Confidence Score: {best_match['score']:.4f} (from matching question: {best_match['Question'][:80]}...)*"
                else:
                    answer_content = "I could not find a highly relevant answer in my knowledge base. Please try rephrasing your question."


                st.markdown(entity_report)
                st.markdown("---")
                st.markdown(answer_content)


                full_assistant_response = f"{entity_report}\n\n---\n\n{answer_content}"
                st.session_state.messages.append({"role": "assistant", "content": full_assistant_response})

else:
    st.warning("Please run `python knowledge_base.py` first to generate the necessary index and database files.")

2025-10-30 08:42:06.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:06.197 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:06.197 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:06.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:06.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:06.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:06.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-30 08:42:06.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar